In [1]:
import sys
sys.path.insert(0, "../torchdriveenv")

In [12]:
!pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 KB 10.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 KB 9.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 KB 13.1 MB/s eta 0:00:00
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.9/208.9 KB 15.1 MB/s eta 0:00:00


In [2]:
import time
import torch
import torchdriveenv
from torch.utils.data import DataLoader

from datasets import TorchDriveEnvEpisodeDataset, DatasetMode
from diffusion import ConditionalDiffusionModel
from model import ConditionalResnet, ConditionalEBMDiffusionModel

/home/kezhang/work/stochastic-critic/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Commercial access denied and fallback to check for academic access.


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
data_dir = "../torchdriveenv/examples/offline_datasets/episode_data_20241025-1300"

In [5]:
dataset = TorchDriveEnvEpisodeDataset(data_dir=data_dir, mode=DatasetMode.action_state)

/home/kezhang/work/stochastic-critic/.venv/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [6]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [7]:
# len(dataset)

In [8]:
batch_size = 64
# data_dim = dataset.state_dim

EMA = .999

n_steps = 100
net_params = {"n_layers": 4,
              "h_dim": 128,
              "emb_dim": 32}

In [9]:
net = ConditionalResnet(n_steps=n_steps, n_layers=4, x_dim=dataset.action_dim, s_dim=dataset.state_dim, h_dim=128, emb_dim=32)

ebm_net = ConditionalEBMDiffusionModel(net)

ddpm = ConditionalDiffusionModel(dataset.action_dim, n_steps, ebm_net, device=device, var_type="beta_forward").to(device)

In [10]:
def logp_unnorm(net, ddpm, x, t, s):
    scale_e = ddpm.energy_scale(-2 - t)
    t = torch.ones((x.shape[0],)).int() * t
    return -net.neg_logp_unnorm(x, t, s) * scale_e

In [11]:
losses = []
test_logpx = []
itr = 0

ddpm.train()
optimizer = torch.optim.Adam(ddpm.parameters(), lr=1e-3)


num_steps = 200000
for itr in range(num_steps):
    ddpm.train()
    
    x, s = next(iter(dataloader))

    start_time = time.time()

    loss = ddpm.loss(x, s)
    loss = loss.mean()
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    duration_update = time.time() - start_time

    if itr % 100 == 0:
        print(itr, loss, "time:", duration_update)
        losses.append(loss)
        
    if itr % 1000 == 0:
        ddpm.eval()
        x_samp = ddpm.sample(s[0], n=1)
        print(x_samp)

0 tensor(0.4514, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.2427520751953125
tensor([[-0.9609,  1.1670]], device='cuda:0', dtype=torch.float64,
       grad_fn=<AddBackward0>)
100 tensor(1.8725, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005777597427368164
200 tensor(0.2619, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011494159698486328
300 tensor(0.9030, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010469436645507812
400 tensor(0.3218, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010483741760253906
500 tensor(0.1691, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010981082916259766
600 tensor(0.0708, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010464191436767578
700 tensor(0.9411, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010430574417114258
800 tensor(0.0581, device='cuda:0'

6900 tensor(0.2954, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010904788970947266
7000 tensor(0.0972, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.006606101989746094
tensor([[-0.4278,  0.4334]], device='cuda:0', dtype=torch.float64,
       grad_fn=<AddBackward0>)
7100 tensor(0.0360, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010554075241088867
7200 tensor(0.0637, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.01051950454711914
7300 tensor(0.0058, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011216878890991211
7400 tensor(0.4339, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011173725128173828
7500 tensor(0.0732, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011348962783813477
7600 tensor(0.4295, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.014950990676879883
7700 tensor(0.1356, dev

13700 tensor(2.4549, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.009989500045776367
13800 tensor(0.5941, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010228395462036133
13900 tensor(0.9553, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.01215982437133789
14000 tensor(0.3709, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.012579202651977539
tensor([[-528.5884, 1668.6140]], device='cuda:0', dtype=torch.float64,
       grad_fn=<AddBackward0>)
14100 tensor(1.2827, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.012748479843139648
14200 tensor(0.8557, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.012215137481689453
14300 tensor(2.0378, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.0063059329986572266
14400 tensor(0.3521, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011409521102905273
14500 tens

20500 tensor(0.1048, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010765790939331055
20600 tensor(0.1932, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011731147766113281
20700 tensor(0.0513, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011223793029785156
20800 tensor(0.0686, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005913972854614258
20900 tensor(1.2984, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.005394935607910156
21000 tensor(0.6329, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.01193380355834961
tensor([[-698.2582, -159.2359]], device='cuda:0', dtype=torch.float64,
       grad_fn=<AddBackward0>)
21100 tensor(0.8512, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011414051055908203
21200 tensor(0.3170, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010582208633422852
21300 tenso

27300 tensor(0.1869, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.012086629867553711
27400 tensor(0.1212, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.01197052001953125
27500 tensor(0.2793, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011270761489868164
27600 tensor(0.1168, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011868953704833984
27700 tensor(0.0617, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011812210083007812
27800 tensor(0.2404, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.01226949691772461
27900 tensor(1.3731, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.01240682601928711
28000 tensor(0.0899, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.012262344360351562
tensor([[-3.1823,  6.0901]], device='cuda:0', dtype=torch.float64,
       grad_fn=<AddBackward0>)
28100 tensor(0.56

...

168700 tensor(0.1161, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.01236724853515625
168800 tensor(11804.2858, device='cuda:0', dtype=torch.float64,
       grad_fn=<MeanBackward0>) time: 0.012178182601928711
168900 tensor(1672.4304, device='cuda:0', dtype=torch.float64,
       grad_fn=<MeanBackward0>) time: 0.013328313827514648
169000 tensor(9.7131, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.01060795783996582
tensor([[-5123.3581, 14854.5487]], device='cuda:0', dtype=torch.float64,
       grad_fn=<AddBackward0>)
169100 tensor(0.6725, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011287689208984375
169200 tensor(79.5411, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011987924575805664
169300 tensor(16.9519, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.01282811164855957
169400 tensor(2.1796, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0

175400 tensor(0.3051, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.012131214141845703
175500 tensor(283.9655, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011610269546508789
175600 tensor(16.2361, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.013698101043701172
175700 tensor(3.3327, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.01242208480834961
175800 tensor(277.3561, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.006181240081787109
175900 tensor(461.3777, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.012277603149414062
176000 tensor(13.5716, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.0107879638671875
tensor([[  869.2350, -2572.2193]], device='cuda:0', dtype=torch.float64,
       grad_fn=<AddBackward0>)
176100 tensor(18.1976, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.0107853412628

182100 tensor(0.4678, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011890649795532227
182200 tensor(0.5569, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.01254582405090332
182300 tensor(0.6198, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.013712167739868164
182400 tensor(0.6086, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.012684106826782227
182500 tensor(0.9366, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.012203454971313477
182600 tensor(0.1422, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.012354373931884766
182700 tensor(0.1825, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.012526273727416992
182800 tensor(0.2577, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.012506723403930664
182900 tensor(0.6756, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.013778448104

188900 tensor(0.6726, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.012401819229125977
189000 tensor(0.0803, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.006746768951416016
tensor([[17.5629, 36.7968]], device='cuda:0', dtype=torch.float64,
       grad_fn=<AddBackward0>)
189100 tensor(0.1035, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.013663530349731445
189200 tensor(0.0736, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.012288570404052734
189300 tensor(0.2484, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.006196498870849609
189400 tensor(0.1687, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.012820243835449219
189500 tensor(0.1692, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.012419939041137695
189600 tensor(0.0617, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.006575822830200195
189700

195700 tensor(0.0207, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010818958282470703
195800 tensor(0.4476, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010887861251831055
195900 tensor(0.0579, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.012646675109863281
196000 tensor(0.5172, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.013301372528076172
tensor([[ -945.4844, -1700.4773]], device='cuda:0', dtype=torch.float64,
       grad_fn=<AddBackward0>)
196100 tensor(0.0669, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.010567188262939453
196200 tensor(0.1016, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.009006500244140625
196300 tensor(0.1044, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.011550188064575195
196400 tensor(0.1345, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>) time: 0.012096881866455078
